![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### ODPi Egeria Hands-On Lab
# Welcome to the UI Asset Search Lab

## Introduction

ODPi Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information (called metadata) about data and the technology that supports it.

In this hands-on lab you will get a chance to have hands-on experience with the UI of Egeria, by searching for assets we add along the way.

## The Scenario

The ODPi Egeria team use the personas and scenarios from the fictitious company called Coco Pharmaceuticals.  (See https://opengovernance.odpi.org/coco-pharmaceuticals/ for more information).

As part of the huge business transformation that Coco Pharmaceuticals has embarked on, they
have created a data lake for managing data for research, analytics, exchange between their internal organizations and business partners (such as hospitals).  As a result, the data lake has to be
designed to handle a wide variety of data, including some highly sensitive and regulated data.

In this lab we use first data engine to import pre-defined assets. Afterwards, the UI where we can search for assets is shown. The main character engaged in this lab is the data scientist named Callie Quartile.

<img style="margin-left: 7%;" src="../images/callie-quartile.png">

> **Callie Quartile:** Data Scientist

## Setting up

Coco Pharmaceuticals make widespread use of ODPi Egeria for tracking and managing their data and related assets.
Figure 1 below shows their metadata servers and the Open Metadata and Governance (OMAG) Server Platforms that are hosting them.  Each metadata server supports a department in the organization.  The servers are distributed across the platform to even out the workload.  Servers can be moved to a different platform if needed.

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

The code below sets up the network addresses for the three platforms.  This varies depending on whether you are running them locally, through **docker-compose** or on **kubernetes**.

In [ ]:
%run ../common/environment-check.ipynb

----
Callie is using the data lake metadata server called `cocoMDS4`. This server is hosted on the Data Lake OMAG Server Platform. It enables business users and the executive team to access data from the data lake.

Check that `cocoMDS4` is running.  

If any of the platforms are not running, follow [this link to set up and run the platform](https://egeria.odpi.org/open-metadata-resources/open-metadata-labs/).  If any server is reporting that it is not configured then
run the steps in the **[Server Configuration](../egeria-server-config.ipynb)** lab to configure
the servers.  Then re-run the previous step to ensure all of the servers are started.

----
## Exercise 1

### Importing IGC events

IBM InfoSphere Information Governance Catalog (IGC) is a tool that allows you to create, manage and share a common business language, document and enact policies and rules and track data lineage.

For this part of the exercises, we have prepared a set of events for Data Engine OMAS in `cocoMDS4` to pick up. The precise process on how these events have been created is more comvoluted, and is out of scope of this lab.

#### Install kafka-python

We will send the IGC events through the kafka bus. The `kafka-python` package allows us to interact with kafka directly with Python.

In [ ]:
!pip install kafka-python

#### Import IGC events

Before importing the events, we first pull the value from an environment variable called `eventBusURLroot` with a default value of `localhost:9092`.  It is used in all of the server configuration documents to connect it to Kafka.

In [ ]:
import os

eventBusURLroot = os.environ.get('eventBusURLroot', 'localhost:9092')

The IGC events to create assets are located in the text file `igc-events.txt`. One line represents one event. The kafka topic we should send these events to is `egeria.omag.openmetadata.repositoryservices.cohort.cocoCohort.OMRSTopic`.

In [ ]:
from kafka import KafkaProducer, KafkaClient

producer = KafkaProducer(bootstrap_servers=eventBusURLroot)

with open('igc-events.txt', 'r') as f:
    for line in f:
        producer.send(
            'egeria.omag.openmetadata.repositoryservices.cohort.cocoCohort.OMRSTopic', 
            str.encode(line)
        )

The code above reads one event at a time, and sends it to the `egeria.omag.openmetadata.repositoryservices.cohort.cocoCohort.OMRSTopic` kafka topic. All OMAS servers listening to this topic will attempt to pick up and process these events. Once successful, we can search for the assets in the UI.

## Exercise 2

### Search for assets in the UI

Now we can go to the UI, located at:
```
https://localhost:18443
```

<img src="../images/egeria-ui-login.png" width="800" height="100" alt="Figure 2" />

> **Figure 2:** Egeria UI's login page

and login with the credential of Callie Quartile:
```
Username: calliequartile
Password: admin
```

---
After logging in, we should end up at the asset search page:

<img src="../images/egeria-ui-asset-search.png" width="800" height="100" alt="Figure 3" />

> **Figure 3:** Egeria UI's asset search page

---

Next, searching for `FNAME` yields the results as follows:

<img src="../images/egeria-ui-asset-list.png" width="800" height="100" alt="Figure 4" />

> **Figure 4:** Egeria UI's asset search result

The following attributes are present:
- **Name**: Name of the asset.
- **Type**: Asset type.
- **Description**: Text description of the asset. In our example this field is empty for all four results.
- **QualifiedName**: Unique identifier to differentiate different assets.
- **Options**: Additional interactive options to interact with the returned assets. The `Lineage` hyperlink is clickable, but asset lineage is not within the scope of this tutorial.